## Utility functions and imports

In [117]:
import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt
import sys
import os
from scipy.fft import dct, idct
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from math import sqrt
import importlib
sys.path.append(os.path.join(os.getcwd(), 'common'))

import reedsolo as rs
importlib.reload(rs)
 

def wpsnr(img1, img2):
  img1 = np.float32(img1)/255.0
  img2 = np.float32(img2)/255.0
  difference = img1-img2
  same = not np.any(difference)
  if same is True:
      return 9999999
  w = np.genfromtxt('csf.csv', delimiter=',')
  ew = convolve2d(difference, np.rot90(w,2), mode='valid')
  decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
  return decibels

def similarity(X, X_star):
    # Computes the similarity measure between the original and the new watermarks.
    norm_X = np.sqrt(np.sum(np.multiply(X, X)))
    norm_X_star = np.sqrt(np.sum(np.multiply(X_star, X_star)))

    if norm_X == 0 or norm_X_star == 0:
        return 0.0  # or return 0 or another appropriate value

    s = np.sum(np.multiply(X, X_star)) / (norm_X * norm_X_star)
    return s

def compute_thr(sim, mark, mark_size=1024, N=1000):
    SIM = np.zeros(N)
    for i in range(N):
        r = np.random.uniform(0.0, 1.0, mark_size)
        SIM[i] = (similarity(mark, r))
    SIMs = SIM.copy()
    SIM.sort()
    t = SIM[-1]
    T = t + (0.1*t)
    print('Threshold: ', T)
    return T, SIMs

In [7]:
#List of different attacks
from sklearn.metrics import roc_curve, auc

import random

#this seed was set just to make you obtain the same result
random.seed(3)
def awgn(img, std, seed):
  mean = 0.0   # some constant
  #np.random.seed(seed)
  attacked = img + np.random.normal(mean, std, img.shape)
  attacked = np.clip(attacked, 0, 255)
  return attacked

def blur(img, sigma):
  from scipy.ndimage.filters import gaussian_filter
  attacked = gaussian_filter(img, sigma)
  return attacked

def sharpening(img, sigma, alpha):
  import scipy
  from scipy.ndimage import gaussian_filter
  import matplotlib.pyplot as plt

  #print(img/255)
  filter_blurred_f = gaussian_filter(img, sigma)

  attacked = img + alpha * (img - filter_blurred_f)
  return attacked

def median(img, kernel_size):
  from scipy.signal import medfilt
  attacked = medfilt(img, kernel_size)
  return attacked

def resizing(img, scale):
  from skimage.transform import rescale
  x, y = img.shape
  attacked = rescale(img, scale)
  attacked = rescale(attacked, 1/scale)
  attacked = attacked[:x, :y]
  return attacked

def jpeg_compression(img, QF):
  from PIL import Image
  img = Image.fromarray(img)
  img = img.convert('L')
  img.save('tmp.jpg',"JPEG", quality=QF)
  attacked = Image.open('tmp.jpg')
  attacked = np.asarray(attacked,dtype=np.uint8)
  os.remove('tmp.jpg')

  return attacked

## Fourth technique

In [8]:
def calculate_parity_bits(data_bits):
    """Calculate the parity bits for a 4-bit data message."""
    p1 = data_bits[0] ^ data_bits[1] ^ data_bits[3]  # Parity for bits 1, 3, 5, 7
    p2 = data_bits[0] ^ data_bits[2] ^ data_bits[3]  # Parity for bits 2, 3, 6, 7
    p3 = data_bits[1] ^ data_bits[2] ^ data_bits[3]  # Parity for bits 4, 5, 6, 7
    return [p1, p2, p3]

def hamming_encode(data_bits):
    """Encode 4 data bits into a 7-bit Hamming (7,4) code."""
    if len(data_bits) != 4:
        raise ValueError("Input must be a list of 4 data bits.")
    
    # Calculate parity bits
    p1, p2, p3 = calculate_parity_bits(data_bits)
    
    # Arrange the codeword: [p1, p2, data1, p3, data2, data3, data4]
    codeword = [p1, p2, data_bits[0], p3, data_bits[1], data_bits[2], data_bits[3]]
    
    return codeword

def split_into_blocks(bit_string, block_size=4):
    """Split the bit string into blocks of size block_size."""
    return [bit_string[i:i + block_size] for i in range(0, len(bit_string), block_size)]

def encode_1024_bit_string(bit_string):
    """Encode a 1024-bit string using Hamming (7,4) encoding in blocks of 4 bits."""
    if len(bit_string) != 1024:
        raise ValueError("Input string must be exactly 1024 bits long.")
    
    # Split the bit string into 4-bit blocks
    blocks = split_into_blocks(bit_string, 4)
    
    # Prepare to store the encoded blocks
    encoded_blocks = []
    
    # Encode each 4-bit block using Hamming (7,4)
    for block in blocks:
        # Convert each 4-bit block (string) into a list of integers
        data_bits = [int(bit) for bit in block]
        
        # Encode the block and add the resulting 7-bit codeword to encoded_blocks
        encoded_blocks.append(hamming_encode(data_bits))
    
    # Flatten the list of encoded blocks into a single list of bits
    encoded_bits = [bit for block in encoded_blocks for bit in block]
    
    # Convert the list of bits into a string
    encoded_string = ''.join(map(str, encoded_bits))
    
    return encoded_string

# Example usage:
bit_string = '110101011010' * 85 + '0' * 4  # Example 1024-bit string (repeated pattern)
encoded_result = encode_1024_bit_string(bit_string)

print(len(encoded_result))
encoded_list = list(encoded_result)
print(len(encoded_list))

for _ in range(512):
    encoded_list[np.random.randint(0, 1023)] = np.random.randint(0, 2)

encoded_result = "".join([str(x) for x in encoded_list])
print(type(encoded_result))

print(f"Original 1024-bit string: {bit_string[:60]}...")  # Print a small portion for brevity
print(f"Encoded string (7,4 Hamming blocks): {encoded_result[:105]}...")  # Print a small portion for brevity
print(f"Encoded length: {len(encoded_result)} bits")

1792
1792
<class 'str'>
Original 1024-bit string: 110101011010110101011010110101011010110101011010110101011010...
Encoded string (7,4 Hamming blocks): 100010100000001011100101111100011011111000101010101001111011000100010101000110100000001111111001011101010...
Encoded length: 1792 bits


In [9]:
def hamming_decode(codeword):
    """Decode the 7-bit Hamming code, correct single-bit errors, and extract the original 4 data bits."""
    if len(codeword) != 7:
        raise ValueError("Codeword must be 7 bits long.")
    
    # Extract the parity and data bits
    p1, p2, d1, p3, d2, d3, d4 = codeword
    
    # Recalculate the parity bits based on the received data
    c1 = p1 ^ d1 ^ d2 ^ d4  # Check parity bit 1
    c2 = p2 ^ d1 ^ d3 ^ d4  # Check parity bit 2
    c3 = p3 ^ d2 ^ d3 ^ d4  # Check parity bit 3
    
    # Combine the parity check results into a single error syndrome
    error_position = (c3 << 2) | (c2 << 1) | c1  # Binary value gives error position
    
    # If error_position is non-zero, correct the error
    if error_position != 0:
        codeword[error_position - 1] ^= 1  # Flip the bit at the error position
    
    # Extract the original data bits from the corrected codeword
    data_bits = [codeword[2], codeword[4], codeword[5], codeword[6]]
    
    return data_bits

def split_into_blocks(bit_string, block_size):
    """Split the bit string into blocks of size block_size."""
    return [bit_string[i:i + block_size] for i in range(0, len(bit_string), block_size)]

def decode_1792_bit_string(encoded_string):
    """Decode a 1792-bit Hamming (7,4) encoded string back into a 1024-bit string."""
    if len(encoded_string) != 1792:
        raise ValueError("Input string must be exactly 1792 bits long.")
    
    # Split the encoded string into 7-bit blocks
    encoded_blocks = split_into_blocks(encoded_string, 7)
    
    # Prepare to store the decoded data bits
    decoded_data_bits = []
    
    # Decode each 7-bit block and extract the original 4 data bits
    for block in encoded_blocks:
        # Convert the 7-bit block (string) into a list of integers
        codeword = [int(bit) for bit in block]
        
        # Decode the block to get the original 4 data bits
        decoded_bits = hamming_decode(codeword)
        
        # Append the decoded 4 data bits to the result
        decoded_data_bits.extend(decoded_bits)
    
    # Convert the list of decoded data bits into a string
    decoded_string = ''.join(map(str, decoded_data_bits))
    
    return decoded_string

# Example usage:
decoded_result = decode_1792_bit_string(encoded_result)

print(f"Decoded 1024-bit string: {decoded_result[:60]}...")  # Print a small portion for brevity
print(f"Decoded length: {len(decoded_result)} bits")
print(decoded_result == bit_string)

print(similarity(np.asarray(list(decoded_result), dtype=np.uint8), np.asarray(list(bit_string), dtype=np.uint8)))

Decoded 1024-bit string: 110100000100111101111000110101011010110110110000011101010010...
Decoded length: 1024 bits
False
0.9121610686310013


In [32]:
def create_perceptual_mask(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    edges = cv2.magnitude(sobel_x, sobel_y)
    mask = cv2.normalize(edges, None, 0, 1, cv2.NORM_MINMAX)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    return mask
 

def modular_alpha(layer, theta, alpha):
    arrayLayer = [1.0, 0.32, 0.16, 0.1]
    arrayTheta = [1, sqrt(2), 1]

    return alpha * arrayLayer[layer] * arrayTheta[theta]

def get_locations(subband):
    sign = np.sign(subband)
    abs_subband = abs(subband)
    locations = np.argsort(-abs_subband, axis=None) # - sign is used to get descending order
    rows = subband.shape[0]
    locations = [(val//rows, val%rows) for val in locations] # locations as (x,y) coordinates

    return abs_subband, sign, locations

def embed_watermark(subband, mark, layer, theta, alpha=0.5, v='multiplicative'):

    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband) 

    watermarked = abs_subband.copy()
    
    for idx, (loc, mark_val) in enumerate(zip(locations[1:], mark)):
        x = locations[idx][0]
        y = locations[idx][1]
        if v == 'additive':
            watermarked[loc] += (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
        elif v == 'multiplicative':
            watermarked[loc] *= 1 + (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
    
    return sign * watermarked

def recursive_embedding(coeffs, mark, alpha, layer, max_layer, v='multiplicative'):
    LL, (LH, HL, HH) = coeffs

    # Base case: if we reach layer 3, embed the watermark and return
    if layer == max_layer:
        watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
        watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
        watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

        watermarked_LL = pywt.idwt2((LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
        return watermarked_LL

    # Recursive case: perform another DWT and recurse
    coeffs_next = pywt.dwt2(LL, 'haar')
    watermarked_LL = recursive_embedding(coeffs_next, mark, alpha, layer + 1, max_layer, v)

    # Embed the watermark at this layer
    watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
    watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
    watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

    # Return the inverse DWT of the watermarked image
    watermarked = pywt.idwt2((watermarked_LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
    return watermarked


def embedding(image, mark, alpha, max_layer=2, v='multiplicative'):
    # Initial wavelet decomposition
    coeffs = pywt.dwt2(image, 'haar')
    

    # mark = "".join([str(x) for x in mark])
    # enc_mark = encode_1024_bit_string(mark)
    # enc_mark = np.asarray(list(enc_mark), dtype=np.uint8)
    # Start recursive embedding from layer 0
    watermarked_image = recursive_embedding(coeffs, mark, alpha, layer=0, max_layer=max_layer, v=v)
    
    return watermarked_image

def extract_watermark(subband, watermarked_subband, layer, theta, alpha=0.5, v='multiplicative'):
    # Create perceptual mask for the subband
    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband)
    abs_watermarked, _, _ = get_locations(watermarked_subband)
    mark_size = 1024
    extracted_mark = np.zeros(mark_size, dtype=np.float64)
    # Loop through each location (except the first one)
    for idx, loc in enumerate(locations[1:mark_size+1]):
        x = locations[idx][0]
        y = locations[idx][1]
        
        if v == 'additive':
            # Reverse the additive watermarking process to extract the mark
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y])
        elif v == 'multiplicative':
            # Reverse the multiplicative watermarking process to extract the mark
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y] * subband[loc])
        
    return np.clip(extracted_mark, 0, 1)

def detect_wm(image, watermarked, alpha, max_layer=2, v='multiplicative'):
    #ori_dct = dct(dct(image,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_or, (LH0_or, HL0_or, HH0_or) = pywt.dwt2(image, 'haar')
    LL1_or, (LH1_or, HL1_or, HH1_or) = pywt.dwt2(LL0_or, 'haar')
    LL2_or, (LH2_or, HL2_or, HH2_or) = pywt.dwt2(LL1_or, 'haar')
     

    #wat_dct = dct(dct(watermarked,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_w, (LH0_w, HL0_w, HH0_w) = pywt.dwt2(watermarked, 'haar')
    LL1_w, (LH1_w, HL1_w, HH1_w) = pywt.dwt2(LL0_w, 'haar')
    LL2_w, (LH2_w, HL2_w, HH2_w) = pywt.dwt2(LL1_w, 'haar')
    
    extracted_wms = []
    w_ex = []
    if max_layer == 2:
        extracted_wms.append(extract_watermark(LH2_or, LH2_w, 2, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL2_or, HL2_w, 2, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH2_or, HH2_w, 2, 1, alpha=alpha, v=v))
        w_ex.append(sum(extracted_wms) / 3)

    extracted_wms = []
    if max_layer >= 1:
        extracted_wms.append(extract_watermark(LH1_or, LH1_w, 1, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL1_or, HL1_w, 1, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH1_or, HH1_w, 1, 1, alpha=alpha, v=v))
        w_ex.append(sum(extracted_wms) / 3)

    extracted_wms = []
    extracted_wms.append(extract_watermark(LH0_or, LH0_w, 0, 0, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HL0_or, HL0_w, 0, 2, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HH0_or, HH0_w, 0, 1, alpha=alpha, v=v))
    w_ex.append(sum(extracted_wms) / 3)

    return w_ex


# def detection(original, watermarked, attacked, alpha, max_layer):
#     w_ex = detect_wm(original, watermarked, alpha, max_layer=max_layer)
#     w_ex_attacked = detect_wm(original, attacked, alpha, max_layer=max_layer)

#     thr = 0.7
#     sim = []
#     decoded_ex_attacked = []

#     for w in w_ex_attacked:
#         w = "".join([str(x) for x in w])
#         decoded_w = decode_1792_bit_string(w)
#         decoded_ex_attacked.append(np.asarray(list(decoded_w), dtype=np.uint8))


#     ex_mark = w_ex[0]
#     ex_mark = "".join([str(x) for x in ex_mark])
#     decoded= decode_1792_bit_string(ex_mark)
#     ex_mark= np.asarray(list(decoded), dtype=np.uint8)



#     for w in decoded_ex_attacked:
#         x = similarity(w, ex_mark)
#         sim.append(x)

#     sim = max(sim)
    
#     if sim >= thr or wpsnr(watermarked, attacked) < 35:
#         return 1
#     return 0


def detection(original, watermarked, attacked, alpha, max_layer):
    ex_mark = detect_wm(original, watermarked, alpha, max_layer=max_layer)
    ex_attacked = detect_wm(original, attacked, alpha, max_layer=max_layer)
    thr = 0.7
    sim = []
    for w in ex_attacked:
        sim.append(similarity(ex_mark[0], w))

    sim = max(sim)
   
    if sim >= thr and wpsnr(watermarked, attacked) > 25:
        return 1
    return 0


In [186]:
enocded_test = None

def create_perceptual_mask(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    edges = cv2.magnitude(sobel_x, sobel_y)
    mask = cv2.normalize(edges, None, 0, 1, cv2.NORM_MINMAX)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    return mask
 

def modular_alpha(layer, theta, alpha):
    arrayLayer = [1.0, 0.32, 0.16, 0.1]
    arrayTheta = [1, sqrt(2), 1]

    return alpha * arrayLayer[layer] * arrayTheta[theta]

def get_locations(subband):
    sign = np.sign(subband)
    abs_subband = abs(subband)
    locations = np.argsort(-abs_subband, axis=None) # - sign is used to get descending order
    rows = subband.shape[0]
    locations = [(val//rows, val%rows) for val in locations] # locations as (x,y) coordinates

    return abs_subband, sign, locations

def embed_watermark(subband, mark, layer, theta, alpha=0.5, v='multiplicative'):

    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband) 

    watermarked = abs_subband.copy()
    
    for idx, (loc, mark_val) in enumerate(zip(locations[1:], mark)):
        x = locations[idx][0]
        y = locations[idx][1]
        if v == 'additive':
            watermarked[loc] += (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
        elif v == 'multiplicative':
            watermarked[loc] *= (1 + (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y]))
    
    return sign * watermarked

def recursive_embedding(coeffs, mark, alpha, layer, max_layer, v='multiplicative'):
    LL, (LH, HL, HH) = coeffs

    # Base case: if we reach layer 3, embed the watermark and return
    if layer == max_layer:
        watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
        watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
        watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

        watermarked_LL = pywt.idwt2((LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
        return watermarked_LL

    # Recursive case: perform another DWT and recurse
    coeffs_next = pywt.dwt2(LL, 'haar')
    watermarked_LL = recursive_embedding(coeffs_next, mark, alpha, layer + 1, max_layer, v)

    # Embed the watermark at this layer
    watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
    watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
    watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

    # Return the inverse DWT of the watermarked image
    watermarked = pywt.idwt2((watermarked_LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
    return watermarked


def embedding(image, mark, alpha, max_layer=2, v='multiplicative'):
    # Initial wavelet decomposition
    coeffs = pywt.dwt2(image, 'haar')
    message = np.packbits(mark).astype(np.uint8)
    mark_in_bytes = message.tobytes()

    splitted_mark = [mark_in_bytes[i:i+8] for i in range(0, len(mark_in_bytes), 8)]
    rsc = rs.RSCodec(8)
    encoded = [rsc.encode(chunk) for chunk in splitted_mark]
    encoded_test = [bytes(list(d)) for d in encoded]
    print(encoded)
    encoded = b''.join(encoded_test)
    encoded = np.frombuffer(encoded, dtype=np.uint8)
    encoded_mark = np.unpackbits(encoded)
    watermarked_image = recursive_embedding(coeffs, encoded_mark, alpha, layer=0, max_layer=max_layer, v=v)
    
    return watermarked_image

def extract_watermark(subband, watermarked_subband, layer, theta, alpha=0.5, v='multiplicative'):
    # Create perceptual mask for the subband
    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband)
    abs_watermarked, _, _ = get_locations(watermarked_subband)
    mark_size = 2048
    encoded_mark = np.zeros(mark_size, dtype=np.float64)
    # Loop through each location (except the first one)
    for idx, loc in enumerate(locations[1:mark_size+1]):
        x = locations[idx][0]
        y = locations[idx][1]
        
        if v == 'additive':
            # Reverse the additive watermarking process to extract the mark
            encoded_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y])
        elif v == 'multiplicative':
            # Reverse the multiplicative watermarking process to extract the mark
            encoded_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y] * subband[loc])
    
    encoded_mark = np.clip(encoded_mark, 0, 1).astype(np.uint8)
    encoded_mark = np.packbits(encoded_mark).astype(np.uint8)
    encoded_mark = encoded_mark.tobytes()
    splitted_encoded_message = [encoded_mark[i:i+16] for i in range(0, len(encoded_mark), 16)]

    return splitted_encoded_message

def detect_wm(image, watermarked, alpha, max_layer=2, v='multiplicative'):
    #ori_dct = dct(dct(image,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_or, (LH0_or, HL0_or, HH0_or) = pywt.dwt2(image, 'haar')
    LL1_or, (LH1_or, HL1_or, HH1_or) = pywt.dwt2(LL0_or, 'haar')
    LL2_or, (LH2_or, HL2_or, HH2_or) = pywt.dwt2(LL1_or, 'haar')
     

    #wat_dct = dct(dct(watermarked,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_w, (LH0_w, HL0_w, HH0_w) = pywt.dwt2(watermarked, 'haar')
    LL1_w, (LH1_w, HL1_w, HH1_w) = pywt.dwt2(LL0_w, 'haar')
    LL2_w, (LH2_w, HL2_w, HH2_w) = pywt.dwt2(LL1_w, 'haar')
    
    extracted_wms = []
    if max_layer == 2:
        extracted_wms.append(extract_watermark(LH2_or, LH2_w, 2, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL2_or, HL2_w, 2, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH2_or, HH2_w, 2, 1, alpha=alpha, v=v))

    if max_layer >= 1:
        extracted_wms.append(extract_watermark(LH1_or, LH1_w, 1, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL1_or, HL1_w, 1, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH1_or, HH1_w, 1, 1, alpha=alpha, v=v))

    extracted_wms.append(extract_watermark(LH0_or, LH0_w, 0, 0, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HL0_or, HL0_w, 0, 2, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HH0_or, HH0_w, 0, 1, alpha=alpha, v=v))

    return extracted_wms



def detection(original, watermarked, attacked, alpha, max_layer):
    ex_mark_encoded = detect_wm(original, watermarked, alpha, max_layer=max_layer)
    ex_attacked_encoded = detect_wm(original, attacked, alpha, max_layer=max_layer)
    thr = 0.7
    sim = []
    rsc = rs.RSCodec(8)

    encoded_mark = ex_mark_encoded[0]

    print(len(ex_mark_encoded))

    decoded_mark = []
    print(similarity(encoded_mark, encoded_test))
    for chunk in encoded_mark:
        rmes, _, _ = rsc.decode(chunk)
        decoded_mark.append(rmes)

    decoded_mark = [bytes(list(d)) for d in decoded_mark]
    decoded_mark = b''.join(decoded_mark)
    decoded_mark = np.frombuffer(decoded_mark, dtype=np.uint8)
    decoded_mark = np.unpackbits(decoded_mark)

    erase_position = []
    for enc_mark in ex_attacked_encoded:
        decoded = []
        for i, chunk in enumerate(enc_mark):
            erase_position = []
            for j, b in enumerate(chunk):
                if encoded_mark[i][j] != b:
                    del chunk[j]
                    erase_position.append(j)
            try:
                rmes, _, _ = rsc.decode(chunk, erase_pos=erase_position)
                decoded.append(rmes)

            except rs.ReedsoloException: 
                decoded.append(b'1'*8)

        decoded = [bytes(list(d)) for d in decoded]
        decoded = b''.join(decoded)
        decoded = np.frombuffer(decoded, dtype=np.uint8)
        decoded = np.unpackbits(decoded)

        sim.append(similarity(decoded_mark, decoded))

    sim = max(sim)
   
    if sim >= thr and wpsnr(watermarked, attacked) > 25:
        return 1
    return 0


In [ ]:
print(np.mean(wpsnr_wm_imgs))

In [187]:
imgs = [cv2.imread(os.path.join('./sample_imgs', img), 0) for img in os.listdir('./sample_imgs')]
mark = np.load('ammhackati.npy')
wm_imgs = []
alpha = 0.88
max_layer = 1


for img in imgs[:10]:
    wm = embedding(img, mark, alpha=alpha, max_layer=max_layer)
    wm_imgs.append(wm)


[bytearray(b'\xfb\xeb\x11\xffO#}\xcfE31fv\x93\xc6"'), bytearray(b'-\xef\xa3\xea\x7fr\x0e\xc0\xc9j#\x9b\x9c\xe0\xfd\xd2'), bytearray(b'\x00LU\x81J\xc6\x8bQ\xc0H\xc7?\xf2\xce\x00\x82'), bytearray(b"\xf1\x0b\x11\'\x16\x05E\x1d\x9e\x92\xb1\xcf\xa2\x1b\xf7\xbb"), bytearray(b'q;\xc7\xb7\xbf?\xb1\nB\xc92\x9d\xca\x88\x19~'), bytearray(b'\x82\x14!kd\xa1\xd5x*&7\x89^=U0'), bytearray(b'\x91\x153:E}\xca\xa2\xa7\xde\xbd\x8e\x1c\x10c\xf8'), bytearray(b'\x19}\xaa\xe4\xd1\x98\x87\x91\x9bu\x95\x9a<V\xc1?'), bytearray(b'D:\x1f{\x9aK\xfco\x9bz\x12\xbf\xb3j\x87J'), bytearray(b'\\\xc4\x95\xe1\xcb\x94\x971\x07\xfe_!b3d\xa7'), bytearray(b'\x84\x95\xac\xcb\xbb\x91{m\x89<\xe9\xd62\r`\x9f'), bytearray(b'p\xddA\xbc\x7f\x88\xa5 \x10\xbb(6BI\x18\x84'), bytearray(b'\x01\x9e\x978j\x95\x88\x83\x92\xd2\xba\xb9*\x8d^~'), bytearray(b'\x04D4\xfd{\x11\x15e\xa7\xfb;\x06\x86\xd3\x1b\xbc'), bytearray(b'\xf6\x08\xe2\xdel\xd6;Ytg\x8dRZ\x08[\xdf'), bytearray(b'\xa7i\xcc\xf51.s\xec\xd8\xf5\xecO\xd5ZF0')]
[bytearray(b'\xfb\xeb\x1

In [192]:
a = [b'\xfb\xeb\x11\xffO#}\xcfE31fv\x93\xc6"', b'-\xef\xa3\xea\x7fr\x0e\xc0\xc9j#\x9b\x9c\xe0\xfd\xd2', b'\x00LU\x81J\xc6\x8bQ\xc0H\xc7?\xf2\xce\x00\x82', b"\xf1\x0b\x11'\x16\x05E\x1d\x9e\x92\xb1\xcf\xa2\x1b\xf7\xbb", b'q;\xc7\xb7\xbf?\xb1\nB\xc92\x9d\xca\x88\x19~', b'\x82\x14!kd\xa1\xd5x*&7\x89^=U0', b'\x91\x153:E}\xca\xa2\xa7\xde\xbd\x8e\x1c\x10c\xf8', b'\x19}\xaa\xe4\xd1\x98\x87\x91\x9bu\x95\x9a<V\xc1?', b'D:\x1f{\x9aK\xfco\x9bz\x12\xbf\xb3j\x87J', b'\\\xc4\x95\xe1\xcb\x94\x971\x07\xfe_!b3d\xa7', b'\x84\x95\xac\xcb\xbb\x91{m\x89<\xe9\xd62\r`\x9f', b'p\xddA\xbc\x7f\x88\xa5 \x10\xbb(6BI\x18\x84', b'\x01\x9e\x978j\x95\x88\x83\x92\xd2\xba\xb9*\x8d^~', b'\x04D4\xfd{\x11\x15e\xa7\xfb;\x06\x86\xd3\x1b\xbc', b'\xf6\x08\xe2\xdel\xd6;Ytg\x8dRZ\x08[\xdf', b'\xa7i\xcc\xf51.s\xec\xd8\xf5\xecO\xd5ZF0']
b = [b'\xfb\xeb\x11\xffO#}\xcfE31fv\x93\xc6"', b'-\xef\x83\xea\x7fr\x0e\xc0\xc9j#\x9b\x9c\xe0}\xd2', b'\x00LU\x81J\xc6\x8bQ\xc0H\xc77\xf2\xce\x00\x82', b"\xf1\x0b\x11'\x16\x05E\x1d\x9e\x92\xb1\xcf\xa2\x1b\xf7;", b'q;\xc7\xb7\xbf?\xb1\nB\xc92\x9dJ\x88\x19~', b'\x82\x14 kD\xa1\xd58*&7\x89V=U0', b'\x91\x153:E}\xca\xa2\xa7\xde\xbd\x8a\x1c\x10c\xf8', b'\x19}\xaa\xe4\xd1\x98\x07\x91\x9bU\x85\x9a<V\xc1?', b'D:\x1f{\x9aK\xfco\x9bz\x12\xbf\xb3j\x87J', b'\x14\xc4\x95a\xcb\x94\x97\x11\x07\xfe]!B3d\xa2', b'\x84\x95\xac\xcb;\x91{m\x89<\xc9\xd62\r`\x9f', b'`\xcdA\xbc\x7f\x08\xa5 \x10;(6B\x08\x18\x84', b'\x01\x9e\x978j\x95\x88\x83\x92\xd2\xba\xb9*\r^~', b'\x04@4\xfd{\x11\x15d\xa7\xfb;\x06\x86\xd3\x1b\xbc', b'\xf2\x08b\x0el\xd2+Ytc\x8dR\x1a\x08K\xdf', b'\xa1)\xccu0,s\xec\x98\xc5\xecO\xd5JB0']


a = b''.join(a)
a = np.frombuffer(a, dtype=np.uint8)
a = np.unpackbits(a)

b = b''.join(b)
b = np.frombuffer(b, dtype=np.uint8)
b = np.unpackbits(b)

print(similarity(b, a))

for c, s in zip(a,b):
    for i, j in zip(c,s):
        if i != j:
            print(i, j)
    print("fine blocco")


0.9738806804565623


TypeError: 'numpy.uint8' object is not iterable

In [188]:
lost = 0
wpsnr_before_attacks = []

wpsnr_after_attacks = []
for wi, img in zip(wm_imgs, imgs[:10]): 
    attacked = jpeg_compression(wi, 80)
    result = detection(img, wi, attacked, alpha=alpha, max_layer=max_layer)
    wpsnr_prev = wpsnr(wi, img)
    wpsnr_after = wpsnr(attacked, wi)

    wpsnr_before_attacks.append(wpsnr_prev)
    
    print(f"WPSNR before the attack: {wpsnr_prev}")
    print(f"WPSNR after the attack: {wpsnr_after}")
    
    if result:
        print("Found")
    else:
        lost += 1
        wpsnr_after_attacks.append(wpsnr_after)
        print("Not found")
    
    print("=========================")

print(f"Mark has been lost {lost}/{len(wm_imgs)} time")
print(f"Avarage WPSNR before attacks {np.mean(wpsnr_before_attacks)}")
print(f"Avarage WPSNR after attacks {np.mean(wpsnr_after_attacks)}")



6


NameError: name 'encoded_test' is not defined

In [66]:
from pyldpc import make_ldpc, encode, decode, get_message


def create_perceptual_mask(subband):

    mask = np.ones(subband.shape)
    mask += compute_brightness_sensitivity(subband) * compute_edge_sensitivity(subband) * compute_texture_sensitivity(subband)
    
    return mask

def compute_brightness_sensitivity(subband):

    # Normalize brightness between 0 and 1
    min_brightness = np.min(subband)
    max_brightness = np.max(subband)
    brightness_sensitivity = (subband - min_brightness) / (max_brightness - min_brightness + 1e-6)
    
    # Invert to give higher sensitivity in dark areas (lower brightness = higher mask value)
    return 1 - brightness_sensitivity

def compute_edge_sensitivity(subband):

    # Compute image gradient (strong edges correspond to higher gradients)
    sobel_x = cv2.Sobel(subband, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(subband, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Normalize gradient magnitude between 0 and 1
    gradient_sensitivity = (gradient_magnitude - np.min(gradient_magnitude)) / (np.max(gradient_magnitude) - np.min(gradient_magnitude) + 1e-6)
    
    return gradient_sensitivity

def compute_texture_sensitivity(subband):
    
    # Compute local variance as a measure of texture
    mean = cv2.blur(subband, (3, 3))
    local_variance = cv2.blur((subband - mean) ** 2, (3, 3))
    
    # Normalize local variance between 0 and 1
    texture_sensitivity = (local_variance - np.min(local_variance)) / (np.max(local_variance) - np.min(local_variance) + 1e-6)
    
    return texture_sensitivity

def modular_alpha(layer, theta, alpha):
    arrayLayer = [1.0, 0.32, 0.16, 0.1]
    arrayTheta = [1, sqrt(2), 1]

    return alpha * arrayLayer[layer] * arrayTheta[theta]

def get_locations(subband):
    sign = np.sign(subband)
    abs_subband = abs(subband)
    locations = np.argsort(-abs_subband, axis=None) # - sign is used to get descending order
    rows = subband.shape[0]
    locations = [(val//rows, val%rows) for val in locations] # locations as (x,y) coordinates

    return abs_subband, sign, locations

def embed_watermark(subband, mark, layer, theta, alpha=0.5, v='multiplicative'):

    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband) 

    watermarked = abs_subband.copy()
    for idx, (loc, mark_val) in enumerate(zip(locations[1:], mark)):
        x = locations[idx][0]
        y = locations[idx][1]
        if v == 'additive':
            watermarked[loc] += (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
        elif v == 'multiplicative':
            watermarked[loc] *= 1 + (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
    
    return sign * watermarked

def recursive_embedding(coeffs, mark, alpha, layer, max_layer, v='multiplicative'):
    LL, (LH, HL, HH) = coeffs

    # Base case: if we reach layer 3, embed the watermark and return
    if layer == max_layer:
        watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
        watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
        watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

        watermarked_LL = pywt.idwt2((LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
        return watermarked_LL

    # Recursive case: perform another DWT and recurse
    coeffs_next = pywt.dwt2(LL, 'haar')
    watermarked_LL = recursive_embedding(coeffs_next, mark, alpha, layer + 1, max_layer, v)

    # Embed the watermark at this layer
    watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
    watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
    watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

    # Return the inverse DWT of the watermarked image
    watermarked = pywt.idwt2((watermarked_LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
    return watermarked


def embedding(image, mark, alpha, max_layer=2, v='multiplicative'):

    n = 2048
    d_v = 4
    d_c = 8
    snr = 20

    _, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
    mark = np.append(mark, [0,0,0])
    encoded_mark = encode(G, mark, snr)

    print(len(encoded_mark))
    # Initial wavelet decomposition
    coeffs = pywt.dwt2(image, 'haar')
    # Start recursive embedding from layer 0
    watermarked_image = recursive_embedding(coeffs, encoded_mark, alpha, layer=0, max_layer=max_layer, v=v)
    
    return watermarked_image


def extract_watermark(subband, watermarked_subband, layer, theta, alpha=0.5, v='multiplicative'):
    # Create perceptual mask for the subband
    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband)
    abs_watermarked, _, _ = get_locations(watermarked_subband)
    mark_size = 2048

    extracted_mark = np.zeros(mark_size, dtype=np.float64)

    # Loop through each location (except the first one)
    for idx, loc in enumerate(locations[1:mark_size+1]):
        x = locations[idx][0]
        y = locations[idx][1]
        
        if v == 'additive':
            # Reverse the additive watermarking process to extract the mark
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y])
        elif v == 'multiplicative':
            # Reverse the multiplicative watermarking process to extract the mark
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / modular_alpha(layer, theta, alpha) * mask[x][y] * subband[loc]


        
    return  np.clip(extracted_mark, 0, 1).astype(np.uint8)

def detect_wm(image, watermarked, alpha, max_layer=2, v='multiplicative'):
    #ori_dct = dct(dct(image,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_or, (LH0_or, HL0_or, HH0_or) = pywt.dwt2(image, 'haar')
    LL1_or, (LH1_or, HL1_or, HH1_or) = pywt.dwt2(LL0_or, 'haar')
    LL2_or, (LH2_or, HL2_or, HH2_or) = pywt.dwt2(LL1_or, 'haar')
     

    #wat_dct = dct(dct(watermarked,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_w, (LH0_w, HL0_w, HH0_w) = pywt.dwt2(watermarked, 'haar')
    LL1_w, (LH1_w, HL1_w, HH1_w) = pywt.dwt2(LL0_w, 'haar')
    LL2_w, (LH2_w, HL2_w, HH2_w) = pywt.dwt2(LL1_w, 'haar')
    
    extracted_wms = []

    if max_layer == 2:
        extracted_wms.append(extract_watermark(LH2_or, LH2_w, 2, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL2_or, HL2_w, 2, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH2_or, HH2_w, 2, 1, alpha=alpha, v=v))
    if max_layer >= 1:
        extracted_wms.append(extract_watermark(LH1_or, LH1_w, 1, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL1_or, HL1_w, 1, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH1_or, HH1_w, 1, 1, alpha=alpha, v=v))

    extracted_wms.append(extract_watermark(LH0_or, LH0_w, 0, 0, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HL0_or, HL0_w, 0, 2, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HH0_or, HH0_w, 0, 1, alpha=alpha, v=v))

    return extracted_wms

def detection(original, watermarked, attacked, alpha, max_layer):
    w_ex = detect_wm(original, watermarked, alpha, max_layer=max_layer)
    w_ex_attacked = detect_wm(original, attacked, alpha, max_layer=max_layer)
    thr = 0.7045
    sim = []
    n = 2048
    d_v = 4
    d_c = 8
    snr = 10
    H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
    ex_mark = []

    w_ex_attacked_decoded = []
    for w in w_ex_attacked:
        decoded = decode(H, w, snr, maxiter=100)
        decoded = get_message(G, decoded)
        decoded = np.delete(decoded, np.s_[-3:])
        w_ex_attacked_decoded.append(decoded)

    # decoded = decode(H, w_ex_attacked[3], snr, maxiter=10)
    # decoded = get_message(G, decoded)
    # decoded = np.delete(decoded, np.s_[-3:])
    # w_ex_att = decoded

    
    decoded = decode(H, w_ex[1], snr)
    decoded = get_message(G, decoded)
    decoded = np.delete(decoded, np.s_[-3:])
    ex_mark = decoded
    
    for w in w_ex_attacked_decoded:
        x = similarity(w, ex_mark)
        sim.append(x)    
    sim = max(sim)

    # sim = similarity(ex_mark, w_ex_att)

    print(sim)
    print("Similarity mark and extracted mark: ", similarity(ex_mark, mark))

    if sim >= thr:
        return 1
    return 0


In [67]:
imgs = [cv2.imread(os.path.join('./sample_imgs', img), 0) for img in os.listdir('./sample_imgs')]
mark = np.load('ammhackati.npy')
wm_imgs = []
alpha = 0.3
max_layer = 1
wpsnr_wm =[]

for img in imgs[:10]:
    wm = embedding(img, mark, alpha=alpha, max_layer=max_layer)
    wm_imgs.append(wm)
    wpsnr_wm.append(wpsnr(img, wm))
    print(wpsnr_wm[-1])


2048
52.97070296877771
2048
60.81587521371497
2048
61.1596742006098
2048
59.757621894466915
2048
56.94168990420093
2048
57.42169689587178
2048
57.553888534908566
2048
51.73778159665592
2048
58.41393258613817
2048
57.27067931150731


In [54]:
attacked = jpeg_compression(wm_imgs[0], 5)
print("avarage wpsnr: ", np.mean(wpsnr_wm))
print(mark)
result = detection(imgs[0], wm_imgs[0], attacked, alpha=alpha, max_layer=max_layer)
if result:
    print("Found")
else:
    print("Not found")

avarage wpsnr:  57.38347234549227
[1 1 1 ... 1 0 0]


/home/sh4ring4n/.conda/envs/OFF_MDS_ENV/lib/python3.8/site-packages/pyldpc/decoder.py:62: UserWarning: Decoding stopped before convergence. You may want
                       to increase maxiter
  warnings.warn("""Decoding stopped before convergence. You may want


0.6893573387803132
Similarity mark and extracted mark:  0.9813067629253163
Not found


In [ ]:
for wi, img in zip(wm_imgs, imgs): 
    attacked = jpeg_compression(wi, 5)
    result = detection(img, wi, attacked, alpha=alpha, max_layer=max_layer)
    print(wpsnr(attacked, wi))
    if result:
        print("Found")
    else:
        print("Not found")

In [50]:
import reedsolo

symbols = 190

def create_perceptual_mask(subband):

    mask = np.ones(subband.shape)
    mask += compute_brightness_sensitivity(subband) * compute_edge_sensitivity(subband) * compute_texture_sensitivity(subband)
    
    return mask

def compute_brightness_sensitivity(subband):

    # Normalize brightness between 0 and 1
    min_brightness = np.min(subband)
    max_brightness = np.max(subband)
    brightness_sensitivity = (subband - min_brightness) / (max_brightness - min_brightness + 1e-6)
    
    # Invert to give higher sensitivity in dark areas (lower brightness = higher mask value)
    return 1 - brightness_sensitivity

def compute_edge_sensitivity(subband):

    # Compute image gradient (strong edges correspond to higher gradients)
    sobel_x = cv2.Sobel(subband, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(subband, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Normalize gradient magnitude between 0 and 1
    gradient_sensitivity = (gradient_magnitude - np.min(gradient_magnitude)) / (np.max(gradient_magnitude) - np.min(gradient_magnitude) + 1e-6)
    
    return gradient_sensitivity

def compute_texture_sensitivity(subband):
    
    # Compute local variance as a measure of texture
    mean = cv2.blur(subband, (3, 3))
    local_variance = cv2.blur((subband - mean) ** 2, (3, 3))
    
    # Normalize local variance between 0 and 1
    texture_sensitivity = (local_variance - np.min(local_variance)) / (np.max(local_variance) - np.min(local_variance) + 1e-6)
    
    return texture_sensitivity

def modular_alpha(layer, theta, alpha):
    arrayLayer = [1.0, 0.32, 0.16, 0.1]
    arrayTheta = [1, sqrt(2), 1]

    return alpha * arrayLayer[layer] * arrayTheta[theta]

def get_locations(subband):
    sign = np.sign(subband)
    abs_subband = abs(subband)
    locations = np.argsort(-abs_subband, axis=None) # - sign is used to get descending order
    rows = subband.shape[0]
    locations = [(val//rows, val%rows) for val in locations] # locations as (x,y) coordinates

    return abs_subband, sign, locations

def embed_watermark(subband, mark, layer, theta, alpha=0.5, v='multiplicative'):

    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband) 

    watermarked = abs_subband.copy()
    for idx, (loc, mark_val) in enumerate(zip(locations[1:], mark)):
        x = locations[idx][0]
        y = locations[idx][1]
        if v == 'additive':
            watermarked[loc] += (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
        elif v == 'multiplicative':
            watermarked[loc] *= 1 + (modular_alpha(layer, theta, alpha) * mark_val * mask[x][y])
    
    return sign * watermarked

def recursive_embedding(coeffs, mark, alpha, layer, max_layer, v='multiplicative'):
    LL, (LH, HL, HH) = coeffs

    # Base case: if we reach layer 3, embed the watermark and return
    if layer == max_layer:
        watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
        watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
        watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

        watermarked_LL = pywt.idwt2((LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
        return watermarked_LL

    # Recursive case: perform another DWT and recurse
    coeffs_next = pywt.dwt2(LL, 'haar')
    watermarked_LL = recursive_embedding(coeffs_next, mark, alpha, layer + 1, max_layer, v)

    # Embed the watermark at this layer
    watermarked_LH = embed_watermark(LH, mark, layer, 0, alpha, v)
    watermarked_HL = embed_watermark(HL, mark, layer, 2, alpha, v)
    watermarked_HH = embed_watermark(HH, mark, layer, 1, alpha, v)

    # Return the inverse DWT of the watermarked image
    watermarked = pywt.idwt2((watermarked_LL, (watermarked_LH, watermarked_HL, watermarked_HH)), 'haar')
    return watermarked


def embedding(image, mark, alpha, max_layer=2, v='multiplicative'):
    rs = reedsolo.RSCodec(symbols)

    encoded_mark = rs.encode(mark.tobytes())
    
    encoded_mark = np.array(encoded_mark)

    encoded_mark = np.clip(encoded_mark, 0, 1).astype(np.uint8)
    
    print(len(encoded_mark))
    # Initial wavelet decomposition
    coeffs = pywt.dwt2(image, 'haar')
    # Start recursive embedding from layer 0
    watermarked_image = recursive_embedding(coeffs, encoded_mark, alpha, layer=0, max_layer=max_layer, v=v)
    
    return watermarked_image


def extract_watermark(subband, watermarked_subband, layer, theta, alpha=0.5, v='multiplicative'):
    # Create perceptual mask for the subband
    mask = create_perceptual_mask(subband)
    abs_subband, sign, locations = get_locations(subband)
    abs_watermarked, _, _ = get_locations(watermarked_subband)
    mark_size = 4064

    extracted_mark = np.zeros(mark_size, dtype=np.float64)

    # Loop through each location (except the first one)
    for idx, loc in enumerate(locations[1:mark_size+1]):
        x = locations[idx][0]
        y = locations[idx][1]
        
        if v == 'additive':
            # Reverse the additive watermarking process to extract the mark
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / (modular_alpha(layer, theta, alpha) * mask[x][y])
        elif v == 'multiplicative':
            # Reverse the multiplicative watermarking process to extract the mark
            # extracted_mark[idx] = ((watermarked_subband[loc] / subband[loc]) - 1) / (modular_alpha(layer, theta, alpha) * mask[x][y])
            extracted_mark[idx] = (watermarked_subband[loc] - subband[loc]) / modular_alpha(layer, theta, alpha) * mask[x][y] * subband[loc]


        
    return  np.clip(extracted_mark, 0, 1).astype(np.uint8)

def detect_wm(image, watermarked, alpha, max_layer=2, v='multiplicative'):
    #ori_dct = dct(dct(image,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_or, (LH0_or, HL0_or, HH0_or) = pywt.dwt2(image, 'haar')
    LL1_or, (LH1_or, HL1_or, HH1_or) = pywt.dwt2(LL0_or, 'haar')
    LL2_or, (LH2_or, HL2_or, HH2_or) = pywt.dwt2(LL1_or, 'haar')
     

    #wat_dct = dct(dct(watermarked,axis=0, norm='ortho'),axis=1, norm='ortho')
    LL0_w, (LH0_w, HL0_w, HH0_w) = pywt.dwt2(watermarked, 'haar')
    LL1_w, (LH1_w, HL1_w, HH1_w) = pywt.dwt2(LL0_w, 'haar')
    LL2_w, (LH2_w, HL2_w, HH2_w) = pywt.dwt2(LL1_w, 'haar')
    
    extracted_wms = []

    if max_layer == 2:
        extracted_wms.append(extract_watermark(LH2_or, LH2_w, 2, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL2_or, HL2_w, 2, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH2_or, HH2_w, 2, 1, alpha=alpha, v=v))
    if max_layer >= 1:
        extracted_wms.append(extract_watermark(LH1_or, LH1_w, 1, 0, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HL1_or, HL1_w, 1, 2, alpha=alpha, v=v))
        extracted_wms.append(extract_watermark(HH1_or, HH1_w, 1, 1, alpha=alpha, v=v))

    extracted_wms.append(extract_watermark(LH0_or, LH0_w, 0, 0, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HL0_or, HL0_w, 0, 2, alpha=alpha, v=v))
    extracted_wms.append(extract_watermark(HH0_or, HH0_w, 0, 1, alpha=alpha, v=v))

    return extracted_wms

def detection(original, watermarked, attacked, alpha, max_layer):
    
    w_ex = detect_wm(original, watermarked, alpha, max_layer=max_layer)
    w_ex_attacked = detect_wm(original, attacked, alpha, max_layer=max_layer)
    
    rs = reedsolo.RSCodec(symbols)
    decoded_mark = rs.decode(w_ex[0])
    ex_mark = np.frombuffer(decoded_mark, dtype=np.uint8)


    thr = 0.7045
    sim = []
    
    for w in w_ex_attacked:
        w = rs.decode(w)
        w = np.frombuffer(w, dtype=np.uint8)
        x = similarity(w, ex_mark)
        sim.append(x)
    
    sim = max(sim)

    print(sim)
    print(similarity(ex_mark,mark))

    if sim >= thr:
        return 1
    return 0




In [49]:
imgs = [cv2.imread(os.path.join('./sample_imgs', img), 0) for img in os.listdir('./sample_imgs')]
mark = np.load('ammhackati.npy')
wm_imgs = []
alpha = 0.3
max_layer = 1
wpsnr_wm =[]

for img in imgs[:10]:
    wm = embedding(img, mark, alpha=alpha, max_layer=max_layer)
    wm_imgs.append(wm)
    wpsnr_wm.append(wpsnr(img, wm))
    print(wpsnr_wm[-1])


4064
49.04086183606544
4064
57.54845150909499
4064
58.03687008886486
4064
56.23778788468766
4064
53.389519797857965
4064
53.48514422155369
4064
54.24430088610031
4064
47.65728897902585
4064
54.30768341203749
4064
53.406553091567616


In [51]:
for wi, img in zip(wm_imgs, imgs): 
    attacked = jpeg_compression(wi, 20)
    result = detection(img, wi, attacked, alpha=alpha, max_layer=max_layer)
    print(wpsnr(attacked, wi))
    if result:
        print("Found")
    else:
        print("Not found")

ReedSolomonError: Could not correct message